In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
def myouter(x):
    return np.outer(x,x)
def gradient(y,X,A,i):
    y_i_hat = N_W_estimator(y,X,A,i)  
    X_j = np.delete(X - X[i,:],i,axis=0)
    X_T_X = np.apply_along_axis(myouter,1,X_j)
    A_T_A = np.dot(A.T,A)
    X_A_T_A_X = np.diag(np.dot(np.dot(X_j,A_T_A),X_j.T))
    K =  norm.pdf( np.sqrt(2 * X_A_T_A_X ) )
    dy = y_i_hat - np.delete(y,i)
    other = np.apply_along_axis(np.multiply,0,X_T_X,K * dy)
    partial_A = 4 * np.dot(A, (y_i_hat - y[i]) * np.sum(other,0))
    return partial_A

def N_W_estimator(y, X, A, i):
    X_j = np.delete(X - X[i,:],i,axis=0)
    A_T_A = np.dot(A.T,A)
    X_A_T_A_X = np.diag(np.dot(np.dot(X_j,A_T_A),X_j.T))

    K =  norm.pdf( np.sqrt(2 * X_A_T_A_X ) )
    molecular = np.sum( K * np.delete(y,i) )
    denominator = np.sum(K)
    y_i_hat = molecular/ denominator
    return y_i_hat
# ##计算梯度时忽略掉那些很小的k_ij
#     if y_i_hat <1e13:
    for j in np.delete(range(n), i):
        X_ij = X[i,:] - X[j,:]
        A_X_ij = np.dot(A, X_ij)
        k_ij = norm.pdf( np.sqrt(2*sum(A_X_ij**2)) )
        sum_k_ij = sum_k_ij + k_ij
        other = other + k_ij * np.einsum('i,j->ij', X_ij, X_ij) * (y_i_hat - y[j])
    partial_A = 4 * np.dot(A , (y_i_hat - y[i] )  * other)
#     else:
#         partial_A = 0
        
    return partial_A


def Loss(y,X,A):
    n = np.shape(X)[0]
    y_hat = [N_W_estimator(y,X,A,i) for i in range(n)]
#     y_hat = np.array(y_hat)
    sh1 = [i for i in range(n) if y_hat[i]<1e13 ] ##舍去那些估计量为零的样本点
    y_hat = np.array(y_hat)
    loss = np.sum((y[sh1] - y_hat[sh1])**2)
    return loss

In [ ]:
def SGD_LR(data_x, data_y, d, maxepochs=500,epsilon=1e-4):
    n, p = np.shape(data_x)
    A_ = np.eye(max(p,d))
    if p >= d:
        A = A_[0:d,:]
    else:
        A = A_[:,0:p]
#     A = np.array([[1,0],[0,1],[1,1]]).T
#     A = np.array([[1,0],[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1]]).T
    alpha_0 = 0.05
    epochs_count = 0
    loss_list = []
    epochs_list = []
    starttime = datetime.datetime.now()
    while (epochs_count < maxepochs ):
        rand_i = np.random.randint(n)  # 随机取一个样本
        loss = Loss(data_y,data_x,A)
#         print(loss)
        grad = gradient(data_y,data_x,A,rand_i) #损失函数的梯度
        A_temp = A - alpha_0 * grad
        loss_new_temp = Loss(data_y,data_x,A_temp)
        if( loss_new_temp - loss < 0 ): #要求迭代之后的损失函数必须下降，否则就重新随机一个样本在进行迭代
            A = A_temp
            loss_new = loss_new_temp
        else:
            loss_new = 1e10  
        if abs(loss_new - loss) < epsilon:
            break
        epochs_list.append(epochs_count)
        epochs_count += 1
    loss = loss_new_temp
    endtime = datetime.datetime.now()  
    time_consump = endtime - starttime
    time_consump = time_consump.seconds
    return [A,loss,time_consump,epochs_count]

In [ ]:
def cv_dimension(K,X,Y):
    n,p = X.shape
    kf = KFold(n_splits = K , shuffle = False, random_state = 12)
    A_list = []
    mean_error_list = []
    param_grid = [2,3]
    i = 0
    for param in param_grid:
        error_list = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index,:] , X[test_index,:]
            Y_train, Y_test = Y[train_index] , Y[test_index] 
            re = SGD_LR(X_train, Y_train,d = param, maxepochs = 50,epsilon=1e-3)
            A = re[0]
            A_A_T = np.dot(A,A.T)
            A_A_T_inv = np.linalg.inv(A_A_T)
            A_P = np.dot( np.dot(A.T,A_A_T_inv), A)
            ###test_error
      
            y_est = [N_W_estimator(Y_test, X_test, A, i) for i in range(len(Y_test))]
            err = sum((Y_test - y_est)**2)
            error_list.append(err)
        mean_error_list.append(np.mean(error_list))
        i+=1
        print(str(i) + 'parameter has been searched')
    indx = mean_error_list.index(min(mean_error_list))
    dimension_hat = param_grid[indx]
            
    return [dimension_hat,mean_error_list]
            
    